## Pipeline: Transient to bronze layer

Here our data will be "copied" to the bronze stage, but before we create a delta table to store data. And then we copy to bronze, maintaining the original data structure.

In [ ]:
from pyspark.sql.functions import col, current_timestamp, input_file_name
from delta.tables import DeltaTable
from notebookutils import mssparkutils

In [ ]:
# Parametro do arquivo de input
input_file = ""

In [ ]:
if not input_file:
    print("Aviso: Parâmetro input_file não recebido.")

In [ ]:
domain = "Licitacoes_Gov"
storage_account = "lablicitacoessa"

transient_path = f"abfss://transient@{storage_account}.dfs.core.windows.net/{domain}/"
bronze_path    = f"abfss://bronze@{storage_account}.dfs.core.windows.net/{domain}/"
archive_path   = f"abfss://archive@{storage_account}.dfs.core.windows.net/{domain}/"

print(f"\n{'='*80}")
print(f"Domínio: {domain}")
print(f"Transient: {transient_path}")
print(f"Bronze: {bronze_path}")
print(f"Archive: {archive_path}")
print(f"{'='*80}\n")

In [ ]:
df_raw = (
    spark.read
    .format("json")
    .load(input_file)
)

In [ ]:
df_bronze = (
    df_raw
    .select(
        col("_airbyte_raw_id").alias("airbyte_id"),
        col("_airbyte_extracted_at").cast("timestamp").alias("data_extracao_airbyte"),
        col("_airbyte_data.*")
    )
    .withColumn("data_processamento_bronze", current_timestamp())
    .withColumn("arquivo_origem", input_file_name())
)


In [ ]:
(
    df_bronze.write
    .format("delta")
    .mode("append")
    .partitionBy("data_extracao_airbyte")
    .save(bronze_path)
)
print("Dados gravados com sucesso na camada bronze")


In [ ]:
try:
    spark.sql(f"""
CREATE TABLE IF NOT EXISTS bronze_{domain.lower()}
USING DELTA
LOCATION '{bronze_path}'
    """)
except Exception as e:
    print(f"\n Erro ao criar tabela bronze_{domain.lower()}: {e}")

In [ ]:
for file_path in jsonl_files:
    file_name = file_path.split("/")[-1]
    fs.mv(
        file_path,
        f"{archive_path}{file_name}"
    )

print("Arquivos movidos para archive com sucesso")